In [9]:
!pip install openpyxl


In [1]:
import pandas as pd
from preprocess import (
    load_and_clean_data,
    select_and_interpolate,
    resample_and_engineer_features,
    split_and_scale
)

from train import train_and_benchmark, walk_forward_validation


2026-01-12 18:16:54.601394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file_path = "Greenhouse-1-Transformed_Data.xlsx"

df_raw= pd.read_excel(
    file_path,
    sheet_name="Sheet1"
)


In [3]:
# File path
df_raw=file_path
# Load and clean
df = load_and_clean_data(df_raw)

# Select sensors and interpolate
df = select_and_interpolate(df)

# Resample and feature engineering
df = resample_and_engineer_features(df, lags=4)

# Split and scale
train_df, val_df, test_df, scaler_X, scaler_y = split_and_scale(df)




/home/sagemaker-user/Ment/preprocess.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(",", np.nan)
/home/sagemaker-user/Ment/preprocess.py:45: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample("1H").mean()


In [4]:
# RQ1: Predictive performance comparison (RMSE & MAE)

results_RQ1 = train_and_benchmark(
    train_df,
    val_df,
    epochs=30,
    batch_size=32
)



Training SVR...
Training RNN...
Training LSTM...
Training GRU...


In [5]:
# Drop timing columns to display only performance metrics
results_RQ1 = results_RQ1.drop(
    columns=["Training Time (s)", "Inference Time (s)"]
)

# Sort by RMSE (descending)
results_RQ1 = results_RQ1.sort_values(
    by="RMSE",
    ascending=False
)

results_RQ1


,Model,RMSE,MAE
0,SVR,0.513161,0.411927
2,LSTM,0.424741,0.377054
1,RNN,0.412758,0.381804
3,GRU,0.391707,0.346883


In [8]:
results_RQ2 = walk_forward_validation(
    data=train_df,
    target_col="Plant height (cm)",
    timesteps=1, # Explicitly set this to avoid NameError
    epochs=30,
    batch_size=32,
    n_folds=5
)
results_RQ2

Evaluating SVR across folds...
Evaluating RNN across folds...
Evaluating LSTM across folds...
Evaluating GRU across folds...


,Model,RMSE Mean,RMSE Std,MAE Mean,MAE Std
0,SVR,0.349973,0.146960,0.289052,0.131912
1,RNN,0.344522,0.128526,0.329964,0.138799
2,LSTM,0.339345,0.140804,0.325527,0.149711
3,GRU,0.333999,0.140810,0.319996,0.149881


In [4]:
# RQ1: Predictive performance comparison (RMSE & MAE)

results_RQ3 = train_and_benchmark(
    train_df,
    val_df,
    epochs=30,
    batch_size=32
)



Training SVR...
Training RNN...
Training LSTM...
Training GRU...


In [5]:
results_RQ3 

,Model,RMSE,MAE,Training Time (s),Inference Time (s)
0,SVR,0.513161,0.411927,0.355854,0.018071
1,RNN,0.404087,0.375093,8.264452,0.363711
2,LSTM,0.422639,0.376554,9.206162,0.372098
3,GRU,0.438824,0.398096,8.216831,0.432868


In [9]:
rq4_df = results_RQ2.copy()


In [10]:
# Define thresholds using quartiles (data-driven, defensible)
rmse_mean_q = rq4_df["RMSE Mean"].quantile([0.33, 0.66])
rmse_std_q  = rq4_df["RMSE Std"].quantile([0.33, 0.66])


In [11]:
def bias_level(rmse):
    if rmse <= rmse_mean_q.iloc[0]:
        return "Low"
    elif rmse <= rmse_mean_q.iloc[1]:
        return "Medium"
    else:
        return "High"

rq4_df["Bias Level"] = rq4_df["RMSE Mean"].apply(bias_level)


In [12]:
def variance_level(std):
    if std <= rmse_std_q.iloc[0]:
        return "Low"
    elif std <= rmse_std_q.iloc[1]:
        return "Medium"
    else:
        return "High"

rq4_df["Variance Level"] = rq4_df["RMSE Std"].apply(variance_level)


In [13]:
rq4_df


,Model,RMSE Mean,RMSE Std,MAE Mean,MAE Std,Bias Level,Variance Level
0,SVR,0.349973,0.146960,0.289052,0.131912,High,High
1,RNN,0.344522,0.128526,0.329964,0.138799,High,Low
2,LSTM,0.339345,0.140804,0.325527,0.149711,Medium,Medium
3,GRU,0.333999,0.140810,0.319996,0.149881,Low,High
